In [15]:
import scipy as sp
import scipy.io
import scipy.signal

import numpy as np

import os
import glob

import itertools

import tqdm.notebook as tqdm

from crosspy.preprocessing.signal import filter_data
from crosspy.core.phase import compute_instantaneous_frequency
from crosspy.core.autocorrelation import get_length_by_cdf

from utils.colornoise import powerlaw_psd_gaussian
# from utils.pac import compute_pac_with_lags

In [3]:
def get_files_grouped(root_path):
    all_files = glob.glob(os.path.join(root_path, '*.mat'))
    files_grouped = dict()
    for group_name, group_files in itertools.groupby(all_files, key=lambda x: os.path.split(x)[~0][:5]):
        files_grouped[group_name] = list(group_files)
        
    return files_grouped

import pickle

In [4]:
f_vals = [2]

while f_vals[~0] < 99:
    f_vals.append(f_vals[~0]*1.05)
    
f_vals = np.array(f_vals)

lags_cycles = np.arange(200)/10

In [5]:
mdd_dir = r'L:\nttk-data3\palva\resting_state\depression\__BB_data\parc2018yeo7_400'
output_path = r'L:\nttk-data3\palva\Vlad\pac_meg_depression_resting_states'
mdd_files = get_files_grouped(mdd_dir)

In [26]:
for subj_name, subj_files in tqdm.tqdm(mdd_files.items()):
    
    output_fname = os.path.join(output_path, subj_name + '_psd.pickle')
    
    if os.path.exists(output_fname):
        continue
    
    subj_psd = np.zeros((len(subj_files), 2049, 400))
    
    for fidx, fname in enumerate(subj_files):
        subj_data = sp.io.loadmat(fname)
        pxx_f, pxx = sp.signal.welch(subj_data['time_series'], fs=1000, nperseg=256*16)
        
        subj_psd[fidx] = pxx.T
    
    output_data = {'frequencies': pxx_f, 'psd': subj_psd}
    pickle.dump(output_data, open(output_fname, 'wb'))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 42/42 [00:00<00:00, 426.11it/s]


In [6]:
sfreq = 1000

import cupy as cp

_pac_kernel = cp.ElementwiseKernel(
             'T x, raw T y, raw I lags, int32 n_cols', 'raw C output',
             '''
             int curr_col = i % n_cols;
             int curr_row = i / n_cols;
             
             int sample_lag = lags[curr_row];
             int offset_idx = curr_col + sample_lag;
             
             if(offset_idx < n_cols) {
                 int compare_idx = curr_row*n_cols + offset_idx;
                 output[i] = x * y[compare_idx];
             }
             else
             {
                 output[i] = __int_as_float(0xFFE00000);
             }
                                       
             ''',
             '_pac_kernel',
            )

def compute_pac_with_lags(data, lag_cycles, signal_frequencies, sr=1000):
    pac_vals = cp.zeros((data.shape[0], len(lag_cycles)))

    ns = data.shape[-1]
    
    data_conj = cp.conj(data)
    phase_diff = data.copy()
    
    for lag_idx, lag in enumerate(lag_cycles):
        lag_samples = cp.rint(lag*sr/signal_frequencies).astype(int)
        
        _pac_kernel(data, data_conj, lag_samples, data.shape[-1], phase_diff)
        pac_vals[:, lag_idx] = cp.abs(cp.nanmean(phase_diff, axis=-1))
    
    return cp.asnumpy(pac_vals)

In [40]:
for subj_name, subj_files in tqdm.tqdm(mdd_files.items()):
    
    output_fname = os.path.join(output_path, subj_name + '_parcels.pickle')
    
#     if os.path.exists(output_fname):
#         continue

    if modification_date(output_fname).year < 2022:
        continue
    
    subj_pacf = np.zeros((len(subj_files), len(f_vals), 400, 200))
    
    for fidx, fname in enumerate(subj_files):
        subj_fdata = sp.io.loadmat(fname)
        for freq_idx, freq in tqdm.tqdm_notebook(enumerate(f_vals), total=len(f_vals),leave=False):
            data_complex = filter_data(subj_fdata['time_series'], 1000, freq, 7.5, n_jobs='cuda')

            data_if = compute_instantaneous_frequency(data_complex, sr=1000).mean(axis=-1)
            data_complex /= cp.abs(data_complex)

            freq_pac = compute_pac_with_lags(data_complex, lags_cycles, data_if, sr=sfreq)
            subj_pacf[fidx, freq_idx] = freq_pac
        
    output_data = {'frequencies': f_vals, 'pac': subj_pacf, 'channel_names': subj_fdata['channel_names']}
    pickle.dump(output_data, open(output_fname, 'wb'))

  0%|                                                                                                                                                       | 0/42 [00:00<?, ?it/s]D:\Vlad\Anaconda3\envs\seeg_meso_env\lib\site-packages\ipykernel_launcher.py:15: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  from ipykernel import kernelapp as app


  0%|          | 0/81 [00:00<?, ?it/s]

 10%|█████████████▎                                                                                                                              | 4/42 [22:32<3:34:09, 338.13s/it]

  0%|          | 0/81 [00:00<?, ?it/s]

 12%|████████████████▋                                                                                                                           | 5/42 [43:48<6:00:47, 585.05s/it]

...

 90%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████             | 38/42 [7:31:33<1:08:12, 1023.23s/it]

  0%|          | 0/81 [00:00<?, ?it/s]

 95%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍      | 40/42 [7:52:40<28:48, 864.32s/it]

  0%|          | 0/81 [00:00<?, ?it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 42/42 [8:13:22<00:00, 704.81s/it]


In [29]:
import os
import datetime
def modification_date(filename):
    t = os.path.getmtime(filename)
    return datetime.datetime.fromtimestamp(t)

from utils.pyutils import reorder

In [8]:
controls_dir = r'L:\nttk-data3\palva\resting_state\controls\__BB_data\parc2018yeo7_400'
controls_output_path = r'L:\nttk-data3\palva\Vlad\pac_meg_controls_rs_omega_7.5_signal_freq'

In [9]:
control_files = get_files_grouped(controls_dir)

In [35]:
for subj_name, subj_files in tqdm.tqdm(control_files.items()):
    output_fname = os.path.join(controls_output_path, subj_name + '_psd.pickle')
    
#     if os.path.exists(output_fname):
#         continue

    subj_psd = np.zeros((len(subj_files), 2049, 400))
    subj_amp = np.zeros((len(subj_files), len(f_vals), 400))
    subj_channel_names = list()
    
    for fidx, fname in enumerate(subj_files):
        subj_data = sp.io.loadmat(fname)
        pxx_f, pxx = sp.signal.welch(subj_data['time_series'], fs=1000, nperseg=256*16)

        subj_psd[fidx] = pxx.T
        
        file_ch_names = [n.strip().split('-')[0] for n in subj_data['channel_names']]
        subj_channel_names.append(file_ch_names)
        
        for freq_idx, freq in tqdm.tqdm(enumerate(f_vals), total=len(f_vals),leave=False):
            data_complex = filter_data(subj_data['time_series'], 1000, freq, 7.5, n_jobs='cuda')

            subj_amp[fidx, freq_idx] = cp.abs(data_complex).mean(axis=-1).get()
            
    
    ref_ch_names = subj_channel_names[0]

    for idx, file_ch_names in enumerate(subj_channel_names):
        subj_psd[idx] = reorder(subj_psd[idx], file_ch_names, ref_ch_names, axis=-1)
        subj_amp[idx] = reorder(subj_amp[idx], file_ch_names, ref_ch_names, axis=-1)
        
    output_data = {'frequencies': pxx_f, 'psd': subj_psd, 'amplitude': subj_amp, 'amp_frequencies': f_vals,
                  'channel_names': ref_ch_names}

    pickle.dump(output_data, open(output_fname, 'wb'))

  0%|          | 0/58 [00:00<?, ?it/s]

  0%|          | 0/81 [00:00<?, ?it/s]

  0%|          | 0/81 [00:00<?, ?it/s]

  0%|          | 0/81 [00:00<?, ?it/s]

  0%|          | 0/81 [00:00<?, ?it/s]

...

  0%|          | 0/81 [00:00<?, ?it/s]

  0%|          | 0/81 [00:00<?, ?it/s]

  0%|          | 0/81 [00:00<?, ?it/s]

  0%|          | 0/81 [00:00<?, ?it/s]

  0%|          | 0/81 [00:00<?, ?it/s]

(12, 2049, 400)

In [91]:
anxiety_dir = r'L:\nttk-data3\palva\resting_state\anxiety\__BB_data\parc2018yeo7_400'
anx_output_path = r'L:\nttk-data3\palva\Vlad\pac_meg_anxiety_resting_states'
anxiety_files = get_files_grouped(anxiety_dir)

In [95]:
for subj_name, subj_files in tqdm.tqdm(anxiety_files.items()):
    
    output_fname = os.path.join(anx_output_path, subj_name + '_psd.pickle')
    
    if os.path.exists(output_fname):
        continue
    
    subj_psd = np.zeros((len(subj_files), 2049, 400))
    
    for fidx, fname in enumerate(subj_files):
        try:
            subj_data = sp.io.loadmat(fname)
        except:
            continue
            
        pxx_f, pxx = sp.signal.welch(subj_data['time_series'], fs=1000, nperseg=256*16)
        
        subj_psd[fidx] = pxx.T
    
    output_data = {'frequencies': pxx_f, 'psd': subj_psd}
    pickle.dump(output_data, open(output_fname, 'wb'))

100%|██████████████████████████████████████████████████████████████████████████████████| 18/18 [01:21<00:00,  4.53s/it]
